In [30]:
from astropy.table import Table
from astropy.io import fits
from astropy import wcs
import os
import re
from shutil import copy
from astropy.cosmology import Planck13, z_at_value
from scipy.interpolate import interp1d, interp2d
from scipy.ndimage import interpolation
import matplotlib.pyplot as plt 
from grizli.multifit import GroupFLT, MultiBeam, get_redshift_fit_defaults
import grizli.utils
import grizli.model
from glob import glob
import grizli.model
import seaborn as sea
import numpy as np
import pandas as pd
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({'xtick.direct'
               'ion': 'in','xtick.top':True,'xtick.minor.visible': True,
               'ytick.direction': "in",'ytick.right': True,'ytick.minor.visible': True})
cmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, light=1.0, as_cmap=True)
from time import time
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

### set home for files
hpath = os.environ['HOME'] + '/'

In [5]:
s_gal_list = glob('/Volumes/Vince_research/Extractions/Updated_Q_gals/stack_specs/s*')
n_gal_list = glob('/Volumes/Vince_research/Extractions/Updated_Q_gals/stack_specs/n*')

In [19]:
sids = [int(re.findall('[0-9]+',os.path.basename(U))[0]) for U in s_gal_list]
nids = [int(re.findall('[0-9]+',os.path.basename(U))[0]) for U in n_gal_list]

In [22]:
files = glob('/Volumes/Vince_research/grism_flts/*flt.fits')
info = grizli.utils.get_flt_info(files)
visits, filters = grizli.utils.parse_flt_files(info=info, uniquename=True)
os.chdir('/Volumes/Vince_research/grism_flts')

ERSPRIME-cxt-20-092.0-F105W 2
ERSPRIME-cxt-21-092.0-F105W 2
ERSPRIME-cxt-19-122.0-F105W 2
ERSPRIME-cxt-22-142.0-F105W 2
ERSPRIME-cxt-23-142.0-F105W 2
ERSPRIME-cxt-20-092.0-G102 4
ERSPRIME-cxt-21-092.0-G102 4
ERSPRIME-cxt-19-122.0-G102 4
ERSPRIME-cxt-22-142.0-G102 4
ERSPRIME-cxt-23-142.0-G102 4
GN1-cxt-47-153.0-F105W 2
GN1-cxt-48-153.0-F105W 2
GN1-cxt-46-173.0-F105W 2
GN1-cxt-49-213.0-F105W 2
GN1-cxt-50-213.0-F105W 2
GN1-cxt-47-153.0-G102 4
GN1-cxt-48-153.0-G102 4
GN1-cxt-46-173.0-G102 4
GN1-cxt-49-213.0-G102 4
GN1-cxt-50-213.0-G102 4
GN2-cxt-55-022.0-F105W 2
GN2-cxt-a4-022.0-F105W 2
GN2-cxt-52-309.0-F105W 2
GN2-cxt-53-309.0-F105W 2
GN2-cxt-51-345.0-F105W 2
GN2-cxt-55-022.0-G102 4
GN2-cxt-a4-022.0-G102 4
GN2-cxt-52-309.0-G102 4
GN2-cxt-53-309.0-G102 4
GN2-cxt-51-345.0-G102 4
GN3-cxt-57-151.0-F105W 2
GN3-cxt-58-151.0-F105W 2
GN3-cxt-56-175.0-F105W 2
GN3-cxt-59-200.0-F105W 2
GN3-cxt-60-200.0-F105W 2
GN3-cxt-57-151.0-G102 4
GN3-cxt-58-151.0-G102 4
GN3-cxt-56-175.0-G102 4
GN3-cxt-59-200.0-G

In [58]:
flds = ['ersprime','gs1','gs2','gs3','gs4','gs5']

for u in flds:
    all_grism_files = []
    for i in range(len(visits)):
        if '0-g1' in visits[i]['product'] and u in visits[i]['product']:
            all_grism_files.extend(visits[i]['files'])
            
    os.chdir('/Volumes/Vince_research/grism_flts')
    grp = GroupFLT(grism_files=all_grism_files, direct_files=[],
                ref_file = hpath + 'Clear_data/goodss_mosaic/goodss_3dhst.v4.0.F125W_orig_sci.fits',
                seg_file = hpath + 'Clear_data/goodss_mosaic/goodss_3dhst.v4.0.F160W_seg.fits',
                catalog = hpath + 'Clear_data/goodss_mosaic/goodss_3dhst.v4.3.cat',
               cpu_count=4)
    
    for i in range(len(sids)):
        os.chdir('/Volumes/Vince_research/grism_beams')
        beams = grp.get_beams(sids[i], size=80)
        if len(beams) > 0:
            mb = MultiBeam(beams, fcontam=1, group_name='s{0}'.format(sids[i]))
            mb.write_beam_fits(verbose=False)
            fls = glob('*{0}*'.format(sids[i]))

            for ii in fls:
                fits.setval(ii, 'EXPTIME', ext=0,value=fits.open(ii)[1].header['EXPTIME'])

Load icxt20f7q.01.GrismFLT.fits!
Load icxt20f8q.01.GrismFLT.fits!


Load icxt20fbq.01.GrismFLT.fits!


Load icxt20fdq.01.GrismFLT.fits!


Load icxt21b7q.01.GrismFLT.fits!
Load icxt21b8q.01.GrismFLT.fits!
Load icxt21bcq.01.GrismFLT.fits!
Load icxt21beq.01.GrismFLT.fits!
Load icxt19o6q.01.GrismFLT.fits!
Load icxt19o7q.01.GrismFLT.fits!
Load icxt19oaq.01.GrismFLT.fits!
Load icxt19ocq.01.GrismFLT.fits!
Load icxt22pmq.01.GrismFLT.fits!
Load icxt22pnq.01.GrismFLT.fits!
Load icxt22pqq.01.GrismFLT.fits!
Load icxt22psq.01.GrismFLT.fits!
Load icxt23pvq.01.GrismFLT.fits!
Load icxt23pwq.01.GrismFLT.fits!
Load icxt23pzq.01.GrismFLT.fits!
Load icxt23q1q.01.GrismFLT.fits!
Files loaded - 95.78 sec.


/Users/vestrada/miniconda3/envs/astroconda/lib/python3.5/site-packages/grizli-0.6.0_115_g3ce5ccf-py3.5-macosx-10.6-x86_64.egg/grizli/model.py:3393: RuntimeWarning: divide by zero encountered in true_divide
  self.ivar = 1/self.grism.data['ERR']**2
/Users/vestrada/miniconda3/envs/astroconda/lib/python3.5/site-packages/grizli-0.6.0_115_g3ce5ccf-py3.5-macosx-10.6-x86_64.egg/grizli/model.py:3422: RuntimeWarning: invalid value encountered in sqrt
  resid = np.abs(self.scif - self.flat_flam)*np.sqrt(self.ivarf)
/Users/vestrada/miniconda3/envs/astroconda/lib/python3.5/site-packages/grizli-0.6.0_115_g3ce5ccf-py3.5-macosx-10.6-x86_64.egg/grizli/model.py:3427: RuntimeWarning: invalid value encountered in sqrt
  contam_mask = ((self.contam*np.sqrt(self.ivar) > contam_sn_mask[0]) &
/Users/vestrada/miniconda3/envs/astroconda/lib/python3.5/site-packages/grizli-0.6.0_115_g3ce5ccf-py3.5-macosx-10.6-x86_64.egg/grizli/model.py:3428: RuntimeWarning: invalid value encountered in sqrt
  (self.model*np.sqrt

Load icxt09f0q.01.GrismFLT.fits!
Load icxt09f2q.01.GrismFLT.fits!
Load icxt09f5q.01.GrismFLT.fits!
Load icxt09f7q.01.GrismFLT.fits!
Load icxt10dbq.01.GrismFLT.fits!
Load icxt10deq.01.GrismFLT.fits!
Load icxt10diq.01.GrismFLT.fits!
Load icxt10dkq.01.GrismFLT.fits!
Load icxt07dxq.01.GrismFLT.fits!
Load icxt07dzq.01.GrismFLT.fits!
Load icxt07e2q.01.GrismFLT.fits!
Load icxt07e4q.01.GrismFLT.fits!
Load icxt08e7q.01.GrismFLT.fits!
Load icxt08e9q.01.GrismFLT.fits!
Load icxt08ecq.01.GrismFLT.fits!
Load icxt08eeq.01.GrismFLT.fits!
Load icxt11qiq.01.GrismFLT.fits!
Load icxt11qkq.01.GrismFLT.fits!
Load icxt11qnq.01.GrismFLT.fits!
Load icxt11qpq.01.GrismFLT.fits!
Load icxt12qvq.01.GrismFLT.fits!
Load icxt12qxq.01.GrismFLT.fits!
Load icxt12r0q.01.GrismFLT.fits!
Load icxt12r2q.01.GrismFLT.fits!
Files loaded - 109.44 sec.
Load icxt03jjq.01.GrismFLT.fits!
Load icxt03jkq.01.GrismFLT.fits!
Load icxt03jnq.01.GrismFLT.fits!
Load icxt03jpq.01.GrismFLT.fits!
Load icxt04e3q.01.GrismFLT.fits!
Load icxt04e4q.0

In [60]:
s_beams = glob('*')

In [66]:
s_beams_id = [int(U.replace('.g102.A.fits','').split('_')[-1]) for U in s_beams]

In [71]:
len(list(set(s_beams_id))),len(sids)

(24, 24)

In [72]:
flds = ['gn1','gn2','gn3','gn4','gn5','gn7']

for u in flds:
    all_grism_files = []
    for i in range(len(visits)):
        if '0-g1' in visits[i]['product'] and u in visits[i]['product']:
            all_grism_files.extend(visits[i]['files'])
            
    os.chdir('/Volumes/Vince_research/grism_flts')
    grp = GroupFLT(grism_files=all_grism_files, direct_files=[],
                ref_file = hpath + 'Clear_data/goodsn_mosaic/goodsn_3dhst.v4.0.F125W_orig_sci.fits',
                seg_file = hpath + 'Clear_data/goodsn_mosaic/goodsn_3dhstP.seg.fits',
                catalog = hpath + 'Clear_data/goodsn_mosaic/goodsn_3dhstP.cat',
               cpu_count=4)
    
    for i in range(len(nids)):
        os.chdir('/Volumes/Vince_research/grism_beams')
        beams = grp.get_beams(nids[i], size=80)
        if len(beams) > 0:
            mb = MultiBeam(beams, fcontam=1, group_name='n{0}'.format(nids[i]))
            mb.write_beam_fits(verbose=False)
            fls = glob('*{0}*'.format(nids[i]))

            for ii in fls:
                fits.setval(ii, 'EXPTIME', ext=0,value=fits.open(ii)[1].header['EXPTIME'])

Load icxt47xcq.01.GrismFLT.fits!
Load icxt47xbq.01.GrismFLT.fits!
Load icxt47xhq.01.GrismFLT.fits!
Load icxt47xfq.01.GrismFLT.fits!


Load icxt48mgq.01.GrismFLT.fits!


Load icxt48mhq.01.GrismFLT.fits!
Load icxt48mnq.01.GrismFLT.fits!
Load icxt48mtq.01.GrismFLT.fits!
Load icxt46swq.01.GrismFLT.fits!
Load icxt46sxq.01.GrismFLT.fits!
Load icxt46t0q.01.GrismFLT.fits!
Load icxt46t2q.01.GrismFLT.fits!
Load icxt49hmq.01.GrismFLT.fits!
Load icxt49hnq.01.GrismFLT.fits!
Load icxt49hqq.01.GrismFLT.fits!
Load icxt49hsq.01.GrismFLT.fits!
Load icxt50hvq.01.GrismFLT.fits!
Load icxt50hwq.01.GrismFLT.fits!
Load icxt50hzq.01.GrismFLT.fits!
Load icxt50i1q.01.GrismFLT.fits!
Files loaded - 70.14 sec.


/Users/vestrada/miniconda3/envs/astroconda/lib/python3.5/site-packages/grizli-0.6.0_115_g3ce5ccf-py3.5-macosx-10.6-x86_64.egg/grizli/model.py:3422: RuntimeWarning: invalid value encountered in sqrt
  resid = np.abs(self.scif - self.flat_flam)*np.sqrt(self.ivarf)
/Users/vestrada/miniconda3/envs/astroconda/lib/python3.5/site-packages/grizli-0.6.0_115_g3ce5ccf-py3.5-macosx-10.6-x86_64.egg/grizli/model.py:3427: RuntimeWarning: invalid value encountered in sqrt
  contam_mask = ((self.contam*np.sqrt(self.ivar) > contam_sn_mask[0]) &
/Users/vestrada/miniconda3/envs/astroconda/lib/python3.5/site-packages/grizli-0.6.0_115_g3ce5ccf-py3.5-macosx-10.6-x86_64.egg/grizli/model.py:3428: RuntimeWarning: invalid value encountered in sqrt
  (self.model*np.sqrt(self.ivar) < contam_sn_mask[1]))
/Users/vestrada/miniconda3/envs/astroconda/lib/python3.5/site-packages/grizli-0.6.0_115_g3ce5ccf-py3.5-macosx-10.6-x86_64.egg/grizli/multifit.py:1221: RuntimeWarning: divide by zero encountered in true_divide
  bea

Load icxt55krq.01.GrismFLT.fits!
Load icxt55ksq.01.GrismFLT.fits!
Load icxt55kvq.01.GrismFLT.fits!
Load icxt55kxq.01.GrismFLT.fits!
Load icxta4mdq.01.GrismFLT.fits!
Load icxta4meq.01.GrismFLT.fits!
Load icxta4mhq.01.GrismFLT.fits!
Load icxta4mjq.01.GrismFLT.fits!
Load icxt52i0q.01.GrismFLT.fits!
Load icxt52i1q.01.GrismFLT.fits!
Load icxt52i4q.01.GrismFLT.fits!
Load icxt52i6q.01.GrismFLT.fits!
Load icxt53i9q.01.GrismFLT.fits!
Load icxt53iaq.01.GrismFLT.fits!
Load icxt53idq.01.GrismFLT.fits!
Load icxt53ifq.01.GrismFLT.fits!
Load icxt51jvq.01.GrismFLT.fits!
Load icxt51jwq.01.GrismFLT.fits!
Load icxt51jzq.01.GrismFLT.fits!
Load icxt51k1q.01.GrismFLT.fits!
Files loaded - 68.68 sec.
Load icxt57aoq.01.GrismFLT.fits!
Load icxt57zyq.01.GrismFLT.fits!
Load icxt57a1q.01.GrismFLT.fits!
Load icxt57aqq.01.GrismFLT.fits!
Load icxt58a9q.01.GrismFLT.fits!
Load icxt58aeq.01.GrismFLT.fits!
Load icxt58b2q.01.GrismFLT.fits!
Load icxt58b4q.01.GrismFLT.fits!
Load icxt56tbq.01.GrismFLT.fits!
Load icxt56tcq.01

In [73]:
n_beams = glob('*')
n_beams_id = [int(U.replace('.g102.A.fits','').split('_')[-1]) for U in n_beams]

In [84]:
45972 in set(n_beams_id)

True

In [86]:
os.chdir('/Users/vestrada/Github/Quiescent_analysis/scripts/')

galDB = pd.read_pickle('../data/sgal_param_DB.pkl')


In [89]:
for i in galDB.index:
    if not galDB.ids[i] in set(n_beams_id):
        print(galDB.ids[i],galDB.SN[i])

17070 5.0
19442 1.2402450905951117
21427 1.6555360482101693
39012 1.4476470742516265
40223 1.9374010869194882
41148 2.073024038452847
41659 1.0623299120165217
46345 1.9685734378336877
48631 6.126496060819549
